In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras import applications
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt

In [3]:
# dimensions of our images.
img_width, img_height = 200, 300

top_model_weights_path = '/home/mksnkv/models/top_tuned/bottleneck_fc_model_documents_5class_inception.h5'
train_data_dir = '/home/mksnkv/Documents/classification/documents_5class_divided/train'
validation_data_dir = '/home/mksnkv/Documents/classification/documents_5class_divided/validation'
evaluation_data_dir = '/home/mksnkv/Documents/classification/documents_5class_divided/evaluation'
nb_train_samples = 3500
nb_validation_samples = 1400
nb_evaluation_samples = 1500
epochs = 20
batch_size = 20

In [4]:
def save_bottlebeck_features():
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    # build the Inception network
    model = applications.InceptionV3(include_top=False, weights='imagenet')

    generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=True)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=True)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

In [13]:
def plot(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [18]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    print(train_data.shape)
    train_labels = np.array(
        [0] * (nb_train_samples // 5) + [1] * (nb_train_samples // 5) + [2] * (nb_train_samples // 5) + [3] * (nb_train_samples // 5) + [4] * (nb_train_samples // 5))
    train_labels = to_categorical(train_labels)
    print(train_labels.shape)
    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    print(validation_data.shape)
    validation_labels = np.array(
        [0] * (nb_validation_samples // 5) + [1] * (nb_validation_samples // 5) + [2] * (nb_validation_samples // 5) + [3] * (nb_validation_samples // 5) + [4] * (nb_validation_samples // 5))
    validation_labels = to_categorical(validation_labels)
    print(validation_labels.shape)
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(5, activation='softmax'))

    custom_adam = optimizers.Adam()
    model.compile(optimizer=custom_adam,
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))

In [7]:
save_bottlebeck_features()

Found 3500 images belonging to 5 classes.
Found 1400 images belonging to 5 classes.


In [ ]:
history = train_top_model()

(3500, 4, 8, 2048)
(3500, 5)
(1400, 4, 8, 2048)
(1400, 5)


In [ ]:
model.save(top_model_weights_path)

In [10]:
plot(model, history)

NameError: name 'model' is not defined

In [ ]:
def evaluation(model):
    datagen = ImageDataGenerator(rescale=1. / 255)
    generator = datagen.flow_from_directory(
        evaluation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    features = model.predict_generator(
        generator, nb_evaluation_samples // batch_size)
    return features

In [ ]:
features = evaluation(model)

In [ ]:
model.test_on_batch(features)

In [ ]:
model.predict_on_batch(features)